In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)


In [2]:
sys.path

['',
 '/home/kai/anaconda3/lib/python36.zip',
 '/home/kai/anaconda3/lib/python3.6',
 '/home/kai/anaconda3/lib/python3.6/lib-dynload',
 '/home/kai/anaconda3/lib/python3.6/site-packages',
 '/home/kai/anaconda3/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/duckling-1.8.0-py3.6.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/JPype1-0.6.3-py3.6-linux-x86_64.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/kai/.ipython',
 '/home/kai/data/shiyi/AlphaBoosting/automl']

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels 

In [6]:
# params
#param = {'trainLen': self.train_len, 'splitCol': 'a', 'col': self.label, 'coefficient': 10, 'n': 2, 'fillna': 22}
# feature_engineering_dict.append({'params': params, 'function': 'count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'unique_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'cumulative_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'reverse_cumulative_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'variance', 'feature_cols': ['a','n']})
# feature_engineering_dict.append({'params': params, 'function': 'count_std_over_mean', 'feature_cols': ['a','b']})


# feature_engineering_dict.append({'params': params, 'function': 'time_to_n_next', 'feature_cols': ['a','t']})
# feature_engineering_dict.append({'params': params, 'function': 'count_in_previous_n_time_unit', 'feature_cols': ['a','t']})
# feature_engineering_dict.append({'params': params, 'function': 'count_in_next_n_time_unit', 'feature_cols': ['a','t']})

# feature_engineering_dict.append({'params': params, 'function': 'woe', 'feature_cols': ['a']})
# feature_engineering_dict.append({'params': params, 'function': 'chi_square', 'feature_cols': ['a']})
# feature_engineering_dict.append({'params': params, 'function': 'mean', 'feature_cols': ['a']})

In [7]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek']#, 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [8]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

#features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count,
#                                      fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
features_to_gen = get_features_to_gen([fe.count])

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['Month', 'DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}}]

In [9]:
categorical_features

['Month', 'DayofMonth', 'DayOfWeek']

In [10]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/' 

In [11]:
from automl_app import logger_config

logger_config.config(project_path+'log_project1.log')

In [12]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'run_record.json' # don't created this file
b = AlphaBoosting(automl_config_file, features_to_gen)

2018-06-15 15:19:12 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #56  | Run record file [./output/run_record.json] not found. Begin the first time run...
2018-06-15 15:19:12 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #89  | save run record
2018-06-15 15:19:12 EST | INFO     | automl_app.app.AlphaBoosting                 | _save_run_record     | #153 | val index is saved at ./log/val_index.json
2018-06-15 15:19:12 EST | INFO     | automl_app.app.AlphaBoosting                 | _save_run_record     | #155 | run record is saved at ./output/run_record.json
2018-06-15 15:19:12 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #93  | generate todo list
2018-06-15 15:19:12 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #109 | feature engineering
2018-06-15 15:19:13 EST | DEBUG    | automl_libs.feature_engineering              | coun

[1]	training's auc: 0.731444
[2]	training's auc: 0.73494
[3]	training's auc: 0.742201
[4]	training's auc: 0.74116
[5]	training's auc: 0.7478
[6]	training's auc: 0.750398


2018-06-15 15:19:27 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #79  | Predicting...


[7]	training's auc: 0.752444
[8]	training's auc: 0.753185


2018-06-15 15:19:27 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #84  | Predictions(1529090360) saved in ./output/gs_saved_preds/.
2018-06-15 15:19:27 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #97  | ./output/project1_grid_search.csv created
2018-06-15 15:19:33 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #73  | [do_preds] is True, generating predictions ...
2018-06-15 15:19:33 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #75  | Retrain model using cv params and all data...


[1]	training's auc: 0.601307
[2]	training's auc: 0.627569
[3]	training's auc: 0.726956
[4]	training's auc: 0.742045
[5]	training's auc: 0.743037


2018-06-15 15:19:34 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #79  | Predicting...
2018-06-15 15:19:34 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #84  | Predictions(1529090368) saved in ./output/gs_saved_preds/.
2018-06-15 15:19:34 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #102 | ./output/project1_grid_search.csv updated


[6]	training's auc: 0.742437
[7]	training's auc: 0.744623
